In [3]:
import requests
import base64
from PIL import Image
import io
import cv2
import numpy as np
import glob
import os
import imutils

def encode(val):
    val=val.replace("+","BGABG")
    val=val.replace("=","AGAAGA")
    val=val.replace("/","OPAWQQQQ")
    return val

def decode(val):
    val=val.replace("BGABG","+")
    val=val.replace("AGAAGA","=")
    val=val.replace("OPAWQQQQ","/")
    return val
def stringToImage(base64_string):
    imgdata = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(imgdata))
# convert PIL Image to an RGB image( technically a numpy array ) that's compatible with opencv
def toRGB(image):
    return cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)


imgs=glob.glob('new test/*.jpg')[1:]
#imgs.append('imgs/21_R.jpg')
for i in imgs:
    name=i.split('\\')[-1]
    print(name)
    img=cv2.imread(i)
    #cv2.imshow('img',img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    h,w,c=img.shape
    if w>500:
        img=imutils.resize(img,width=500)
    _,buffer=cv2.imencode('.jpg',img)
    img_encoded=base64.b64encode(buffer)
    sttt=(str(img_encoded)[2:])[:-1]
    e=encode(sttt)
    res=requests.get(f"http://127.0.0.1:5000/knee?image={e}")

    img_trans=decode(res.json()["img_st"])
    img_trans_2=decode(res.json()["img_st_2"])
    img_osteophytes=decode(res.json()['osteophytes'])
    
    img_transparent = toRGB(stringToImage(img_trans))
    img_transparent_2=toRGB(stringToImage(img_trans_2))
    img_osteophytes_img=toRGB(stringToImage(img_osteophytes))
    
    #cv2.imwrite('img_trans.jpg',img_transparent)
    cv2.imshow('img',cv2.add(img,img_transparent))
    cv2.imshow('img_2',cv2.add(img,img_transparent_2))
    cv2.imshow('osteophytes',cv2.add(img,img_osteophytes_img))
    cv2.imshow('img_total',cv2.add(cv2.add(cv2.add(img,img_transparent),img_transparent_2),img_osteophytes_img))
    cv2.imwrite(f'{name}',cv2.add(cv2.add(cv2.add(img,img_transparent),img_transparent_2),img_osteophytes_img))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

002 GRADE I.jpg
003 grade III.jpg
004 GRADE I.jpg
